In [21]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import math


%matplotlib inline

plt.rc('axes', labelsize=20) 
plt.rcParams["figure.figsize"] = (10,10)
plt.rc('xtick', labelsize=13)
plt.rc('ytick', labelsize=13) 
mpl.rcParams['axes.titlesize'] = 24

In [22]:
from numpy import math

def fill_names(line, merged, not_merged):
    if not isinstance(line['candidate_name'], basestring):
        correct_name = not_merged.ix[not_merged.INE_ID == line.INE_ID]['candidate_name']
        if len(correct_name) == 1:
            merged.set_value(col='candidate_name', index=line.name, value=correct_name.iloc[0])
            
            len_before = len(not_merged)
            not_merged.drop(not_merged.index[not_merged.INE_ID == line.INE_ID], inplace=True)
            len_after = len(not_merged)
            assert len_before - len_after == 1
        else:
            if len(correct_name) > 1:
                print 'More than one citizens party in INE_ID ' + str(line.INE_ID)
            if len(correct_name) == 0:
                print 'Zero with citizens party in INE_ID ' + str(line.INE_ID)
                
def fill_ballot_order(line, merged, not_merged):
    if math.isnan(line['Order']):
        correct_ballot_order = not_merged.ix[not_merged.INE_ID == line.INE_ID]['Order']
        if len(correct_ballot_order) == 1:
            merged.set_value(col='Order', index=line.name, value=correct_ballot_order.iloc[0])
            
            len_before = len(not_merged)
            not_merged.drop(not_merged.index[not_merged.INE_ID == line.INE_ID], inplace=True)
            len_after = len(not_merged)
            assert len_before - len_after == 1
            
        else:
            if len(correct_ballot_order) > 1:
                print 'More than one citizens party in INE_ID ' + str(line.INE_ID)
            if len(correct_ballot_order) == 0:
                print 'Zero with citizens party in INE_ID ' + str(line.INE_ID)

# 2013

In [23]:
local_election_results_2013 = pd.read_csv('data/processed_data/local_elections_2013_results.csv', encoding='utf-8')
local_election_candidates_2013 = pd.read_csv('data/processed_data/local_elections_candidates_2013.csv', encoding='utf-8')
local_elections_order_2013 = pd.read_csv('data/processed_data/ballot-order-local-elections-2013.csv')

In [24]:
merged_results_candidates_2013 = pd.merge(
    local_election_results_2013, local_election_candidates_2013, 
    left_on=['INE_ID', 'Sigla', 'year'], right_on=['INE_ID', 'party', 'year'],
    how='outer')
merged_results_candidates_2013 = merged_results_candidates_2013.drop('party', 1)

len_everything_together = len(merged_results_candidates_2013)

not_merged_results_candidates_2013 = merged_results_candidates_2013[merged_results_candidates_2013.Concelho.isnull()]
merged_results_candidates_2013 = merged_results_candidates_2013[merged_results_candidates_2013.Concelho.notnull()]

assert len_everything_together == (len(not_merged_results_candidates_2013) + len(merged_results_candidates_2013))

In [25]:
# Some hacking to merge the rest of the entries
for i in range(len(merged_results_candidates_2013)):
    line = merged_results_candidates_2013.ix[i]
    fill_names(line, merged_results_candidates_2013, not_merged_results_candidates_2013)
    
assert len(not_merged_results_candidates_2013) == 0

In [26]:
merged_2013 = pd.merge(
    merged_results_candidates_2013, local_elections_order_2013, 
    left_on=['INE_ID', 'Sigla'], right_on=['INE_ID', 'Party'],
    how='outer'
)

len_everything_together = len(merged_2013)

not_merged_2013 = merged_2013[merged_2013.Concelho.isnull()]
merged_2013 = merged_2013[merged_2013.Concelho.notnull()]

assert len_everything_together == (len(not_merged_2013) + len(merged_2013))

In [27]:
# Some hacking to merge the missing ballot order numbers
for i in range(len(merged_2013)):
    line = merged_2013.ix[i]
    fill_ballot_order(line, merged_2013, not_merged_2013)
    
# assert len(not_merged_2013) == 0

## There is a problem with the data. Here is the e-mail I've send to CNE (Nacional Comission of Elections) in hope to try to solve them

Estou a analisar dados das eleições autárquicas de 2013 e deparei-me com algo estranho. Estou-me a referir apenas a dados referentes a eleições para a Câmara Municipal.

No documento do sorteio das candidaturas (disponível aqui) estão presentes alguns partidos que depois não aparecem nos resultados das eleições (disponíveis aqui).

Por exemplo, segundo o documento do sorteio, para Ferreira do Zêzere, o partido B.E. supostamente estaria em primeiro lugar no boletim de voto. Mas nos resultados, este partido nem sequer aparece listado. 

Este problema acontece com os seguintes locais e partidos:

[TABLE]

A primeira coluna pode ser ignorada. INE_ID é o número de identificação dado ao local pelo Instituto Nacional de Estatística.

Porque é que isto acontece?

In [28]:
# Mark the Locations where the problem ocurrs 
merged_2013['problems_ballot_order'] = merged_2013.INE_ID.isin(not_merged_2013.INE_ID)

# 2009

In [62]:
local_election_results_2009 = pd.read_csv('data/processed_data/local_elections_2009_results.csv', encoding='utf-8')
local_election_candidates_2009 = pd.read_csv('data/processed_data/local_elections_candidates_2009.csv', encoding='utf-8')

In [64]:
merged_results_candidates_2009 = pd.merge(
    local_election_results_2009, local_election_candidates_2009, 
    left_on=['INE_ID', 'Sigla', 'year'], right_on=['INE_ID', 'party', 'year'],
    how='outer')
merged_results_candidates_2009 = merged_results_candidates_2009.drop('party', 1)

len_everything_together = len(merged_results_candidates_2009)

not_merged_results_candidates_2009 = merged_results_candidates_2009[merged_results_candidates_2009.Concelho.isnull()]
merged_results_candidates_2009 = merged_results_candidates_2009[merged_results_candidates_2009.Concelho.notnull()]

assert len_everything_together == (len(not_merged_results_candidates_2009) + len(merged_results_candidates_2009))

In [65]:
# Some hacking to merge the rest of the entries
for i in range(len(merged_results_candidates_2009)):
    line = merged_results_candidates_2009.ix[i]
    fill_names(line, merged_results_candidates_2009, not_merged_results_candidates_2009)
    
assert len(not_merged_2009) == 0

In [66]:
merged_2009 = merged_results_candidates_2009

# Join data form all the years into just one data frame

In [51]:
elections_df = pd.concat([merged_2009, merged_2013])

In [52]:
import gender_guesser.detector as gender
gender_detector = gender.Detector()

def get_gender(name):  
    first_name = name.split(' ')[0]
    
    male_names = [u'João', u'Acílio', u'Litério', u'Hersílio', u'Agusto', u'Romão', u'Hélder', u'Dulcídio', u'Alano'
                  u'Gerónimo', u'Isaltino', u'Avantino', u'Atílio', u'Vitor', u'Beraldino', u'Estevão', u'Hernani',
                  u'Fábio', u'Juvenálio', u'Edegar', u'Bráulio', u'Vasques', u'Joviano', u'Alano', u'Gerónimo',
                  u'Honório', u'Albérico', u'Tomé', u'Gualter', u'Flamiano', u'Milcíades', u'Cílio', u'Parcidio',
                  u'Herlander', u'Lélio', u'Alírio', u'Patrique', u'Dinarte', u'Dírio', u'Jesus', u'Orlindo']
    female_names = [u'Sílvia', u'Ercília', u'Urãnia', u'Zuraida', u'Fermelinda', u'Brizelinda', u'Léli', u'Nair']
    
    if first_name in male_names:
        return u'male'
    
    if first_name in female_names:
        return u'female'
    
    return gender_detector.get_gender(first_name)


In [53]:
elections_df['gender'] = elections_df['candidate_name'].apply(get_gender)

In [54]:
elections_df.ix[elections_df.gender == u'unknown']

,Abstention (%),Brancos,Concelho,INE_ID,Inscritos,Mandatos,Nulos,Order,Party,Sigla,Tipo,Votantes,Votos,Votos (%),candidate_name,problems_ballot_order,year,gender


In [55]:
elections_df.gender.unique()

array([u'male', u'female'], dtype=object)

In [56]:
elections_df

,Abstention (%),Brancos,Concelho,INE_ID,Inscritos,Mandatos,Nulos,Order,Party,Sigla,Tipo,Votantes,Votos,Votos (%),candidate_name,problems_ballot_order,year,gender
0,39.038067,454.0,ÁGUEDA,101,43870.0,4.0,253.0,NaN,NaN,PS,NaN,26744.0,14569.0,NaN,Gil Nadais Resende Da Fonseca,NaN,2009.0,male
1,39.038067,454.0,ÁGUEDA,101,43870.0,3.0,253.0,NaN,NaN,PPD/PSD,NaN,26744.0,9133.0,NaN,Manuel De Castro Azevedo,NaN,2009.0,male
2,39.038067,454.0,ÁGUEDA,101,43870.0,0.0,253.0,NaN,NaN,PND,NaN,26744.0,105.0,NaN,Alexandra Isabel Ferreira Abrantes,NaN,2009.0,female
3,39.038067,454.0,ÁGUEDA,101,43870.0,0.0,253.0,NaN,NaN,PCP-PEV,NaN,26744.0,509.0,NaN,Nelson Manuel Nunes Leal,NaN,2009.0,male
4,39.038067,454.0,ÁGUEDA,101,43870.0,0.0,253.0,NaN,NaN,B.E.,NaN,26744.0,342.0,NaN,Vítor Manuel Aguiar Gomes,NaN,2009.0,male
5,39.038067,454.0,ÁGUEDA,101,43870.0,0.0,253.0,NaN,NaN,CDS-PP,NaN,26744.0,1379.0,NaN,José Carlos Esmeraldo Dos Santos,NaN,2009.0,male
6,39.093574,218.0,ALBERGARIA-A-VELHA,102,22175.0,0.0,149.0,NaN,NaN,PCP-PEV,NaN,13506.0,245.0,NaN,Bruno Miguel Alves Bento,NaN,2009.0,male
7,39.093574,218.0,ALBERGARIA-A-VELHA,102,22175.0,2.0,149.0,NaN,NaN,CDS-PP,NaN,13506.0,4378.0,NaN,António Augusto Amaral Loureiro E Santos,NaN,2009.0,male
8,39.093574,218.0,ALBERGARIA-A-VELHA,102,22175.0,4.0,149.0,NaN,NaN,PPD/PSD,NaN,13506.0,6452.0,NaN,João Agostinho Pinto Pereira,NaN,2009.0,male
9,39.093574,218.0,ALBERGARIA-A-VELHA,102,22175.0,1.0,149.0,NaN,NaN,PS,NaN,13506.0,1836.0,NaN,Jesus Manuel Vidinha Tomás,NaN,2009.0,male
